# ONT PromethION RUN at ANU using Q20 Kits
# M7xNui

### Run info:
Run Parameters
Flow Cell Type FLO-PRO112
Kit SQK-Q20EA


### ANU DATA HERE:
https://storage.powerplant.pfr.co.nz/input/genomic/plant/Vaccinium/corymbosum/20220208_M7xNui/20220208_1226_2A_PAI81355_0aef349b/
ANU did the basecalling with GuppyV5 sup accuracy. Run report here: https://storage.powerplant.pfr.co.nz/input/genomic/plant/Vaccinium/corymbosum/20220208_M7xNui/20220208_1226_2A_PAI81355_0aef349b/report_PAI81355_20220208_1232_0aef349b.pdf

## Guppy v6 Duplex basecalling


In [12]:
WKDIR=/workspace/hraijc/BB_M7xNui_Assembly
BASECALLDIR=/workspace/hraijc/BB_M7xNui_Assembly/GuppyV6_basecalling
QCDIR=/workspace/hraijc/BB_M7xNui_Assembly/ONT_QC/

In [13]:
cd $WKDIR

In [3]:
mkdir log

In [14]:
module load pfr-python3

In [5]:
#Need to add the guppy basecaller to the path, not sure how to get around this.
export PATH="/home/hraijc/.local/bin:/workspace/hraijc/packages/guppyV6/ont-guppy/bin:$PATH"

In [8]:
mkdir ${WKDIR}/fast5

In [9]:
#Many Fast5 directories. Need to combine into one.
bsub -J cpfast5_1  -o $WKDIR/log/cpfast5_1.out -e $WKDIR/log/cpfast5_1.err -n 1 \
"cp /input/genomic/plant/Vaccinium/corymbosum/20220208_M7xNui/20220208_1226_2A_PAI81355_0aef349b/fast5_skip/* ${WKDIR}/fast5"

Job <730841> is submitted to default queue <lowpriority>.


In [10]:
bsub -J cpfast5_2  -o $WKDIR/log/cpfast5_2.out -e $WKDIR/log/cpfast5_2.err -n 1 \
"cp /input/genomic/plant/Vaccinium/corymbosum/20220208_M7xNui/20220208_1226_2A_PAI81355_0aef349b/fast5_pass/* ${WKDIR}/fast5"

Job <730842> is submitted to default queue <lowpriority>.


In [11]:
bsub -J cpfast5_3  -o $WKDIR/log/cpfast5_3.out -e $WKDIR/log/cpfast5_3.err -n 1 \
"cp /input/genomic/plant/Vaccinium/corymbosum/20220208_M7xNui/20220208_1226_2A_PAI81355_0aef349b/fast5_fail/* ${WKDIR}/fast5"

Job <730843> is submitted to default queue <lowpriority>.


In [12]:
#Make sure to delete when done.
du -sh ${WKDIR}/fast5

707G	/workspace/hraijc/BB_M7xNui_Assembly/fast5


In [6]:
/home/hraijc/.local/bin/guppy_duplex -h

usage: guppy_duplex [-h] [--basecaller_exe BASECALLER_EXE]
                    [--duplex_basecaller_exe DUPLEX_BASECALLER_EXE]
                    --input_path INPUT_PATH --save_path SAVE_PATH
                    [--disable_logging] [--skip_simplex] [--skip_duplex]
                    [--call_non_duplex_reads] [--device DEVICE]
                    [--do_read_splitting]
                    [--duplex_chunks_per_runner DUPLEX_CHUNKS_PER_RUNNER]
                    [--recursive]

All-in-one processing of r10.4 duplex reads using guppy

optional arguments:
  -h, --help            show this help message and exit
  --basecaller_exe BASECALLER_EXE
                        Path to the guppy_basecaller executable. If
                        unspecified it is assumed that the basecaller exe is
                        in the path.
  --duplex_basecaller_exe DUPLEX_BASECALLER_EXE
                        Path to the guppy_basecaller_duplex executable. If
                        unspecified it is assum

In [13]:
#Calling all fast5 files. Simplex and duplex. non_duplex_reads with GuppyV6 'sup' accuracy
bsub -n 9 -J guppyM7Nui_basecalling1 -o $WKDIR/log/guppyM7Nui_basecalling1.out -e $WKDIR/log/guppyM7Nui_basecalling1.err -R 'select[defined(gpu)] rusage[mem=40000]' \
"guppy_duplex \
--input_path ${WKDIR}/fast5 \
--save_path ${WKDIR}/GuppyV6_basecalling \
--device 'cuda:0' \
--do_read_splitting \
--duplex_chunks_per_runner 1200 \
--call_non_duplex_reads"


Job <730849> is submitted to default queue <lowpriority>.


## Comparing quality of reads with Guppyv5 and Guppyv6 Duplex

### Using MinIONQC

In [15]:
#ONT QC
module load R/4.0.0
#Rscript /workspace/hraijc/git_clones/MinIONQC.R -h


Loading R/4.0.0
  Loading requirement: unixODBC/2.3.0 JAGS/4.2.0 gdal/2.4.0 proj/5.2.0


In [1]:
mkdir -p /workspace/hraijc/BB_M7xNui_Assembly/ONT_QC/ONT_QC_GuppyV6_duplex
mkdir -p /workspace/hraijc/BB_M7xNui_Assembly/ONT_QC/ONT_QC_GuppyV6_simplex
mkdir -p /workspace/hraijc/BB_M7xNui_Assembly/ONT_QC/ONT_QC_GuppyV6_simplexduplex



In [16]:
#QC Guppy_duplex
bsub -J minqct  -o $WKDIR/log/minqct.out -e $WKDIR/log/minqct.err -n 1 \
"Rscript /workspace/hraijc/git_clones/MinIONQC.R -i ${BASECALLDIR}/final/duplex/sequencing_summary.txt -o ${QCDIR}/ONT_QC_GuppyV6_duplex -c TRUE"


Job <732612> is submitted to default queue <lowpriority>.


In [17]:
#QC Guppy_simplex
bsub -J minqct2 -o $WKDIR/log/minqct2.out -e $WKDIR/log/minqct2.err -n 1 \
"Rscript /workspace/hraijc/git_clones/MinIONQC.R -i ${BASECALLDIR}/final/simplex/sequencing_summary.txt -o ${QCDIR}/ONT_QC_GuppyV6_simplex -c TRUE"



Job <732613> is submitted to default queue <lowpriority>.


In [6]:
bsub -I "cat ${BASECALLDIR}/final/simplex/sequencing_summary.txt >> ${BASECALLDIR}/sim_dup_seq_sum.txt"

Job <732607> is submitted to default queue <lowpriority>.
<<Waiting for dispatch ...>>
<<Starting on aklppg34>>


In [3]:
wc -l ${BASECALLDIR}/final/duplex/sequencing_summary.txt

80721 /workspace/hraijc/BB_M7xNui_Assembly/GuppyV6_basecalling/final/duplex/sequencing_summary.txt


In [7]:
bsub -I "tail -n 80720 ${BASECALLDIR}/final/duplex/sequencing_summary.txt >> ${BASECALLDIR}/sim_dup_seq_sum.txt"

Job <732608> is submitted to default queue <lowpriority>.
<<Waiting for dispatch ...>>
<<Starting on aklppg34>>


In [10]:
bsub -I "grep 'filename' ${BASECALLDIR}/sim_dup_seq_sum.txt"

Job <732611> is submitted to default queue <lowpriority>.
<<Waiting for dispatch ...>>
<<Starting on wkoppb43>>
filename	read_id	run_id	batch_id	channel	mux	start_time	duration	num_events	passes_filtering	template_start	num_events_template	template_duration	sequence_length_template	mean_qscore_template	strand_score_template	median_template	mad_template	scaling_median_template	scaling_mad_template


In [18]:
#QC Guppy_duplex+Guppy_Simplex
bsub -J minqct3 -o $WKDIR/log/minqct3.out -e $WKDIR/log/minqct3.err -n 1 \
"Rscript /workspace/hraijc/git_clones/MinIONQC.R -i ${BASECALLDIR}/sim_dup_seq_sum.txt -o ${QCDIR}/ONT_QC_GuppyV6_simplexduplex -c TRUE"


Job <732614> is submitted to default queue <lowpriority>.


In [24]:
module unload R/4.0.0
module unload pfr-python3

### Using PycoQC

In [31]:
ml load pycoqc

In [32]:
bsub -J pycoQC -o $WKDIR/log/pycoQC.out -e $WKDIR/log/pycoQC.err -n 1 \
"pycoQC -f ${BASECALLDIR}/final/simplex/sequencing_summary.txt ${BASECALLDIR}/final/duplex/sequencing_summary.txt \
    -o ${QCDIR}/BB_M7xNui_ONTreport_GuppyV6Duplex.html \
    --report_title 'BB_M7xNui_GuppyV6Duplex' \
    --min_pass_qual 10" 


Job <732681> is submitted to default queue <lowpriority>.


In [27]:
bsub -J pycoQC2 -o $WKDIR/log/pycoQC2.out -e $WKDIR/log/pycoQC2.err -n 1 \
"pycoQC -f ${BASECALLDIR}/final/duplex/sequencing_summary.txt -o ${QCDIR}/BB_M7xNui_ONTreport_GuppyV6_DuplexONLY.html --report_title 'BB_M7xNui_GuppyV6_DuplexONLY' --min_pass_qual 10" 


Job <732616> is submitted to default queue <lowpriority>.
